In [3]:
import json
import numpy as np
import pandas as pd
import pickle



import sqlite3

In [4]:
with open("cfg.json", "r") as jin:
    cfg = json.loads(jin.read())

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [5]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["install_db"])
cursor_proj = conn_proj.cursor()

query_p = """
SELECT *

FROM grouped_policy

"""


grouped_df = pd.read_sql_query(query_p, conn_proj)


conn_proj.close
grouped_df.sort_values(by = ['zipcode','month']).head()

,month,zipcode,i_count,size,price,discount,housing,p_type_31,p_type_32,p_type_33,p_type_37,p_type_46,p_type_50,p_type_78,p_type_87,p_type_88,p_type_92,p_type_86
0,3.0,00501,0.0,0.0,11644.84,1506.85,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
12184,6.0,00501,0.0,0.0,11519.12,2793.05,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
24368,7.0,00501,0.0,0.0,11477.38,3206.44,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
36552,8.0,00501,0.0,0.0,11435.74,3612.23,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
48736,9.0,00501,0.0,0.0,11394.17,4010.49,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [6]:

demo_df = pd.read_pickle(cfg["data_dir"] + cfg["zip_stats"])
demo_df.head()

,zipcode,med_income,med_home_value,home_count,pop_density,NREL_PSM_2019,cost_electricity
0,01001,58733.0,213000.0,7557.0,1466.0,4.575282,21.92
1,01002,54422.0,338900.0,10388.0,528.0,4.593613,21.92
3,01005,68644.0,208500.0,2044.0,115.0,4.677823,21.92
4,01007,71875.0,260000.0,5839.0,278.0,4.559018,21.92
5,01008,71635.0,247200.0,586.0,23.0,4.607418,21.92


In [7]:
g_df = grouped_df.merge(demo_df, left_on = 'zipcode', right_on = 'zipcode', how = 'left').dropna()

In [8]:
g_df['housing'] = g_df.housing.fillna(g_df.home_count)

In [9]:
g_df['count_per_home'] = g_df.i_count / g_df.housing

In [10]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["install_db"])
cursor_proj = conn_proj.cursor()

g_df.to_sql("grouped_df", conn_proj, if_exists="replace", index=False)

conn_proj.commit

<function Connection.commit>